In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl

from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline


import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
pd.set_option('display.max_rows', 1000)

mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

sns.set(style = "darkgrid")

In [ ]:
V_url = "https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_doses_admin_global.csv"
df_Vacc = pd.read_csv(V_url)
df_Vacc.head()

In [ ]:
df_Vacc['Country_Region'].nunique()

In [ ]:
total_shots = df_Vacc[df_Vacc['Country_Region'] == 'India']['2022-06-21'].max()
total_popln = df_Vacc[df_Vacc['Country_Region'] == 'India']['Population'].max()
total_shots/total_popln
# print(0.5*168248674/145200000)
# print(0.5*168248674/145200000)

In [ ]:
V_countries = ['Italy', 'US', 'Spain', 'Germany', 'Korea, South']

In [ ]:
time_index = df_Vacc.columns[12:]
df_Vplot = pd.DataFrame({'date':time_index})
df_Vplot

In [ ]:
for each in V_countries:
    df_Vplot[each] = np.array(df_Vacc[df_Vacc['Country_Region'] == each].iloc[:, 12::].max())
df_Vplot[each]

In [ ]:
df_Vplot

In [ ]:
for each in V_countries:
    df_Vplot[each+" % vaccinated"] =(100/3)*(df_Vplot[each])/(df_Vacc[df_Vacc['Country_Region']== each]['Population'].max())
# Considering 3-doeses as a metric of complete vaccination
df_Vplot

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df_Vplot)

In [ ]:
df_Vplot.interpolate(method = 'linear', limit_direction = 'backward', inplace = True)
msno.matrix(df_Vplot)

In [ ]:
fig = go.Figure()

# definition of plotting individual trace
for each in V_countries:
    fig.add_trace(go.Scatter(x = df_Vplot.date, 
                             y = df_Vplot[each+" % vaccinated"], 
                             mode = 'markers+lines',
                             opacity =0.9,
                             name =each, 
                             line_width = 2, 
                             marker_size =4
                            )
                 )
    
# layout inputs
fig.update_layout(xaxis_title = "Time", 
                  yaxis_title = "Percentage_of_Vaccinated_Population (source johns hopkins csse, log-scale",
                  width = 1000, 
                  height = 500)

# fig.update_yaxes(type = 'log', range = [1.1, 9])
# fig.update_layout(xaxis_rangeslider_visible = True)
# fig.show(renderer = 'chrome')
fig.show()

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()

# Defines layout of the dashboard
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
   
    dcc.Dropdown(
        id='country_drop_down',
        options=[
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Spain', 'value': 'Spain'},
            {'label': 'Germany', 'value': 'Germany'},
            {'label': 'Korea,South', 'value': 'Korea,South'}
        ],
        value=['US', 'Germany'], # which are pre-selected
        multi=True
    ),   
        
    dcc.Graph(figure=fig, id='main_window_slope')
])

In [ ]:
from dash.dependencies import Input, Output

#Make dashboard interactive
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    traces = [] 
    for each in country_list:
        traces.append(dict(x=df_Vplot.date,
                                y=df_Vplot[each+" % vaccinated"],
                                mode='markers+lines',
                                opacity=0.8,
                                line_width=1,
                                marker_size=3, 
                                name=each
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                yaxis_title="Percentage of Vaccinated people (source johns hopkins csse, log-scale)",
                xaxis={'title':'Timeline',
                        'tickangle':-30,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                        
                      },
                yaxis={'title':'Percentage of Vaccinated people (source johns hopkins csse, log-scale)',
                       'type':"log",
                       'range':'[1.1,5.5]'
                      }
        )
    }

In [ ]:
app.run_server(debug=True, use_reloader=False)